In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import load_model

In [9]:
kerasModels = {
    'mlp': 'serialized/keras/mlpModel.keras',
    'lstm': 'serialized/keras/lstmModel.keras',
    'dnn': 'serialized/keras/dnnModel.keras'
}

# Loading a serialized model
model = load_model(kerasModels['dnn'])

In [10]:
# New dataset for forecasting
data = {
    'Location': ["Bengaluru", "ITO, Delhi", "New Ind Town, Faridabad"],
    'Year': [2023, 2023, 2022],
    'Month': [2, 2, 10],
    'Day': [23, 22, 2],
    'Hour': [8, 14, 10],
    'PM2.5': [43, 500, 185],
    'PM10': [78, 480, 199],
    'O3': [26, 91, 10],
    'CO': [258, 78, 52],
    'SO2': [10, 17, 12],
    'NO2': [17, 47, 26]
}
new_data = pd.DataFrame(data)
data = pd.DataFrame(data)
new_data = data[['Location', 'Day', 'Hour', 'PM2.5', 'PM10', 'O3', 'CO', 'SO2', 'NO2']].copy()

In [11]:
# Label encoding
le = LabelEncoder()
new_data['Location'] = le.fit_transform(new_data['Location'])
new_data['Hour'] = le.fit_transform(new_data['Hour'])

# Data pre-processing
new_data.fillna(new_data.mean(), inplace=True)

# Convert hours to numeric format
new_data['Hour'] = new_data['Hour'].astype(str).str.split(":").str[0].astype(int)

# Feature selection
X_new = new_data[['Location','Day', 'Hour', 'PM2.5', 'PM10', 'O3', 'CO', 'SO2', 'NO2']]

In [12]:
# Data scaling (use the same scaler as during training)
scaler = MinMaxScaler()
X_new_scaled = scaler.fit_transform(X_new)

# Transformation for LSTM (add dimension for time series)
X_new_scaled = np.expand_dims(X_new_scaled, axis=2)

In [13]:
# Prognostication
predictions = model.predict(X_new_scaled)

print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
[[9.9586594e-01 1.0178132e-04 4.0323571e-03 9.7550501e-10 1.1250074e-09
  1.8824392e-15]
 [1.5045563e-14 5.6696251e-16 2.1671427e-04 5.6307808e-10 9.9963081e-01
  1.5237476e-04]
 [4.5334447e-05 2.2126244e-04 8.4819980e-03 9.8548317e-01 1.2745372e-03
  4.4938005e-03]]


In [14]:
# Decoding of forecasts (classes)
predicted_classes = np.argmax(predictions, axis=1)

# Displaying categories
category_mapping = {
    '0': 'Good',
    '1': 'Moderate',
    '2': 'USG', 
    '3': 'Unhealthy',
    '4': 'Very Unhealthy',
    '5': 'Severe'
}
predicted_categories = [category_mapping[str(cls)] for cls in predicted_classes]

print("Input data:")
print(X_new)
print("\nProbabilities for each class:")
print(predictions)
print("\nPredicted classes:")
print(predicted_classes)
print("\nPredicted categories:")
print(predicted_categories)

Input data:
   Location  Day  Hour  PM2.5  PM10  O3   CO  SO2  NO2
0         0   23     0     43    78  26  258   10   17
1         1   22     2    500   480  91   78   17   47
2         2    2     1    185   199  10   52   12   26

Probabilities for each class:
[[9.9586594e-01 1.0178132e-04 4.0323571e-03 9.7550501e-10 1.1250074e-09
  1.8824392e-15]
 [1.5045563e-14 5.6696251e-16 2.1671427e-04 5.6307808e-10 9.9963081e-01
  1.5237476e-04]
 [4.5334447e-05 2.2126244e-04 8.4819980e-03 9.8548317e-01 1.2745372e-03
  4.4938005e-03]]

Predicted classes:
[0 4 3]

Predicted categories:
['Good', 'Very Unhealthy', 'Unhealthy']
